# Waste Classification Training (Colab Ready)

Notebook này hướng dẫn toàn bộ pipeline: chuẩn bị dữ liệu, huấn luyện mô hình CNN/Transfer Learning, đánh giá và tạo demo Gradio.

## 1. Thiết lập môi trường
- Chạy trên GPU (Runtime → Change runtime type → GPU)
- Kết nối Google Drive nếu dữ liệu/model lưu trên đó.

In [ ]:
#@title Mount Google Drive (tùy chọn)
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#@title Cài đặt phụ thuộc
!pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
!pip install -q albumentations==1.4.7 timm==1.0.3 grad-cam==1.5.5 gradio==4.44.0 scikit-learn==1.5.2

## 2. Chuẩn bị dự án
- Nếu đã upload project lên GitHub, clone trực tiếp.
- Nếu làm việc trên Drive, unzip project vào `/content/TrashProject`.

In [ ]:
#@title Clone hoặc đồng bộ mã nguồn
import os, shutil, zipfile, sys

PROJECT_PATH = "/content/TrashProject"  # chỉnh sửa nếu cần
GIT_REPO_URL = "https://github.com/<your-account>/TrashProject.git"  # TODO: cập nhật

if not os.path.exists(PROJECT_PATH):
    !git clone $GIT_REPO_URL $PROJECT_PATH
else:
    print(f"Sử dụng thư mục có sẵn: {PROJECT_PATH}")

sys.path.append(PROJECT_PATH)
os.chdir(PROJECT_PATH)

## 3. Tải và tiền xử lý dữ liệu
- Có thể dùng script sẵn có để tải TrashNet về thư mục data/raw.
- Hoặc dùng dataset công khai khác/Kaggle ở các cell tiếp theo.
- Có thể thay bằng dataset tự thu thập (đưa vào thư mục data/raw).


In [ ]:
#@title Tải TrashNet từ GitHub (tùy chọn)
import os
DOWNLOAD_TRASHNET = False  #@param {type:"boolean"}
DATA_ROOT = "/content/TrashProject/data"  #@param {type:"string"}
RAW_DATA_DIR = f"{DATA_ROOT}/raw"
os.makedirs(RAW_DATA_DIR, exist_ok=True)
if DOWNLOAD_TRASHNET:
    !python scripts/download_trashnet.py --output-dir {RAW_DATA_DIR}
    print("TrashNet downloaded.")
print(f"Raw data folder: {RAW_DATA_DIR}")


In [ ]:
#@title Tải TrashNet từ Kaggle (tùy chọn)
# Yêu cầu tạo ~/.kaggle/kaggle.json trước khi chạy
DOWNLOAD_DATA = False  #@param {type:"boolean"}
DATA_ROOT = "/content/TrashProject/data"  #@param {type:"string"}

os.makedirs(DATA_ROOT, exist_ok=True)

if DOWNLOAD_DATA:
    !kaggle datasets download -d asdasdasasdas/garbage-classification
    with zipfile.ZipFile("garbage-classification.zip", "r") as zf:
        zf.extractall(DATA_ROOT)
    os.remove("garbage-classification.zip")

print(f"Data folder: {DATA_ROOT}")

In [ ]:
#@title Tạo train/val/test split (70/20/10)
DATA_ROOT = globals().get('DATA_ROOT', '/content/TrashProject/data')
from pathlib import Path
import random
import shutil

raw_dir = Path(DATA_ROOT) / "raw"
train_dir = Path(DATA_ROOT) / "train"
val_dir = Path(DATA_ROOT) / "val"
test_dir = Path(DATA_ROOT) / "test"

for folder in [train_dir, val_dir, test_dir]:
    folder.mkdir(parents=True, exist_ok=True)

if raw_dir.exists():
    for class_dir in raw_dir.iterdir():
        if not class_dir.is_dir():
            continue
        images = list(class_dir.glob("*"))
        random.shuffle(images)
        n = len(images)
        n_train = int(0.7 * n)
        n_val = int(0.2 * n)
        splits = {
            train_dir / class_dir.name: images[:n_train],
            val_dir / class_dir.name: images[n_train:n_train + n_val],
            test_dir / class_dir.name: images[n_train + n_val:],
        }
        for split_dir, split_imgs in splits.items():
            split_dir.mkdir(parents=True, exist_ok=True)
            for img_path in split_imgs:
                shutil.copy(img_path, split_dir / img_path.name)
    print("Hoàn thành chia tập dữ liệu.")
else:
    print("Bỏ qua bước chia tập vì không tìm thấy data/raw.")

## 4. Cấu hình và huấn luyện
Sử dụng lớp `WasteTrainer` trong `src/training/trainer.py`.

In [ ]:
#@title Khởi tạo cấu hình huấn luyện
from pathlib import Path

# Hyperparameters
MODEL_NAME = "resnet18"  #@param ["resnet18", "mobilenetv3", "efficientnetb0"]
EPOCHS = 15  #@param {type:"integer"}
IMG_SIZE = 224  #@param {type:"integer"}
BATCH_SIZE = 32  #@param {type:"integer"}
NUM_WORKERS = 2  #@param {type:"integer"}
LOSS = "focal"  #@param ["cross_entropy", "focal"]
FOCAL_GAMMA = 2.0  #@param {type:"number"}
OPTIM = "adamw"  #@param ["adam", "adamw", "sgd"]
LR = 3e-4  #@param {type:"number"}
WEIGHT_DECAY = 1e-4  #@param {type:"number"}
SCHEDULER = "onecycle"  #@param ["onecycle", "cosine", "step", "none"]
MAX_LR = 1e-3  #@param {type:"number"}
DEVICE = "cuda"  #@param ["cuda", "cpu"]
LOG_EVERY = 20  #@param {type:"integer"}
FREEZE_BACKBONE_EPOCHS = 5  #@param {type:"integer"}
OUTPUT_DIR = "artifacts"  #@param {type:"string"}

from src.training.dataset import DataConfig
from src.training.losses import LossConfig
from src.training.optim import OptimConfig, SchedulerConfig
from src.training.trainer import TrainConfig, WasteTrainer

data_cfg = DataConfig(
    train_dir=Path(DATA_ROOT) / "train",
    val_dir=Path(DATA_ROOT) / "val",
    test_dir=Path(DATA_ROOT) / "test",
    img_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKERS,
)

loss_cfg = LossConfig(name=LOSS, gamma=FOCAL_GAMMA) if LOSS == "focal" else LossConfig(name=LOSS)
optim_cfg = OptimConfig(name=OPTIM, lr=LR, weight_decay=WEIGHT_DECAY)
scheduler_cfg = SchedulerConfig(name=SCHEDULER if SCHEDULER != "none" else None, max_lr=MAX_LR)

train_cfg = TrainConfig(
    data=data_cfg,
    loss=loss_cfg,
    optim=optim_cfg,
    scheduler=scheduler_cfg,
    epochs=EPOCHS,
    model_name=MODEL_NAME,
    freeze_backbone_epochs=FREEZE_BACKBONE_EPOCHS,
    output_dir=Path(OUTPUT_DIR),
    device=DEVICE,
    log_every=LOG_EVERY,
)

trainer = WasteTrainer(train_cfg)


In [ ]:
#@title Bắt đầu huấn luyện
report, cm = trainer.train()
report

In [ ]:
#@title Hiển thị ma trận nhầm lẫn
import matplotlib.pyplot as plt
import seaborn as sns
import torch

if cm is not None:
    cm_np = cm.numpy() if isinstance(cm, torch.Tensor) else cm
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm_np, annot=True, fmt="d", cmap="Blues", xticklabels=trainer.idx_to_class.values(), yticklabels=trainer.idx_to_class.values())
    plt.xlabel("Predicted")
    plt.ylabel("True")
    plt.show()
else:
    print("Chưa có tập test để vẽ confusion matrix.")

## 5. Demo Gradio
Tải trọng số tốt nhất và tạo UI đơn giản cho phép người dùng upload ảnh.

In [ ]:
#@title Khởi tạo Gradio demo
import gradio as gr
from torchvision import transforms
from PIL import Image
import torch

model = trainer.model
checkpoint = torch.load(trainer.config.output_dir / "best.pt", map_location=trainer.device)
model.load_state_dict(checkpoint["model_state"])
model.eval()

preprocess = transforms.Compose(
    [
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]
)

class_names = [trainer.idx_to_class[idx] for idx in sorted(trainer.idx_to_class)]

def predict(image: Image.Image):
    tensor = preprocess(image).unsqueeze(0).to(trainer.device)
    with torch.no_grad():
        logits = model(tensor)
        probs = torch.softmax(logits, dim=1).cpu().numpy()[0]
    return {class_names[i]: float(probs[i]) for i in range(len(class_names))}

gr.Interface(fn=predict, inputs=gr.Image(type="pil"), outputs=gr.Label(num_top_classes=3)).launch(share=False, show_api=False)


## 6. Lưu kết quả và tải xuống
- Lưu lại mô hình, báo cáo metrics vào Drive.
- Đảm bảo cập nhật báo cáo đồ án với bảng kết quả và nhận xét.